# 1.Introduction

In this notebook I used huge data set with complete 'geonames' which contains all stored by humans spots in the world. **Next, I ask three questions:**
* "**How are the biggest cities distributed?**" - in chapter 3 I give the answer what are the trends in the large cities distribution
* "**What is continent according to DBSCAN?**" - the question from the title is answered in the chapter 4. This topic is probably the most interesting in this notebook as the definition of continent remains very discussable
* "**How to approximate elevation and what we can read from it?**" - in this last chapter 5 I try to properly answer the question how to approximate the elvationa nd plot it

# 2.Data preparation

First, I introduce useful libraries:

In [ ]:
import numpy as np
import pandas as pd
import random
import os
import matplotlib.pyplot as plt
import seaborn as sns 
import math
import netCDF4
import gc
from scipy.interpolate import griddata
from osgeo import gdal
from mpl_toolkits.basemap import Basemap
from sklearn.preprocessing import StandardScaler
from math import cos, asin, sqrt
from numpy import nansum,nanmean,linspace,meshgrid
from numpy import meshgrid
from sklearn.cluster import DBSCAN
from matplotlib.colors import Normalize
%matplotlib inline

Random_seed=123

Second, I run the code to clean data set from [my other notebook](https://www.kaggle.com/jjmewtw/total-geo-analysis-spatial-cleaning-plotting):

In [ ]:
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a))

def round_up_to_even(f):
    return math.ceil(f / 2.) * 2

n = 11061987
s = 11061987
skip = sorted(random.sample(range(n),n-s))
        
df_path = "../input/geonames-database/geonames.csv"
df = pd.read_csv(df_path,skiprows=skip,index_col="geonameid")
ISO = pd.read_csv('../input/alpha-country-codes/Alpha__2_and_3_country_codes.csv', sep=';')
Population = pd.read_csv('../input/population-by-country-2020/population_by_country_2020.csv')

C = (df.dtypes == 'object')
CategoricalVariables = list(C[C].index)
Integer = (df.dtypes == 'int64') 
Float   = (df.dtypes == 'float64') 
NumericVariables = list(Integer[Integer].index) + list(Float[Float].index)

df=df.drop(['alternatenames','admin2 code','admin3 code','admin4 code','cc2'], axis=1)

df["latitude_app"] = df.apply(lambda row: round_up_to_even(row['latitude']),axis=1)
df["longitude_app"] = df.apply(lambda row: round_up_to_even(row['longitude']),axis=1)

elevation_table = df[['elevation','latitude_app','longitude_app']].groupby(['latitude_app',
                'longitude_app']).agg({'elevation': lambda x: x.mean(skipna=True)}).sort_values(by=['latitude_app', 
                'longitude_app'], ascending=False).reset_index()

df = pd.merge(df,  elevation_table,  on =['latitude_app', 'longitude_app'],  how ='inner')

WorldAverageElevation = 840
df['elevation_y']=df['elevation_y'].fillna(WorldAverageElevation)

df=df.drop(['elevation_x'], axis=1)

ISO['Country'] = ISO.apply(lambda row: str.rstrip(row['Country']),axis=1)

ISO_toMerge = ISO.drop(['Alpha-3 code','Numeric'], axis=1)
ISO_toMerge=ISO_toMerge.rename(columns={"Alpha-2 code": "country code"})
df = pd.merge(df, ISO_toMerge,  on ='country code',  how ='inner')

df_sample = df.sample(n=100000,random_state=Random_seed)

Aggregated = df[['name','Country']]
Aggregated = Aggregated.groupby(['Country']).agg(['count']).sort_values([('name', 'count')], ascending=False)
Aggregated['Percentage'] = round(Aggregated[['name']] / df.shape[0],2)
Aggregated.columns = Aggregated.columns.get_level_values(0)
Aggregated.columns = [''.join(col).strip() for col in Aggregated.columns.values]

Population = Population.rename(columns={"Country (or dependency)": "Country"})

Population[['Country']] = Population[['Country']].replace("Czech Republic (Czechia)", "Czechia")
Population[['Country']] = Population[['Country']].replace("United States", "United States of America")
Population[['Country']] = Population[['Country']].replace("United Kingdom", "United Kingdom of Great Britain and Northern Ireland")
Population[['Country']] = Population[['Country']].replace("Vietnam", "Viet Nam")
Population[['Country']] = Population[['Country']].replace("Laos", "Lao People Democratic Republic")
Population[['Country']] = Population[['Country']].replace("State of Palestine", "Palestine")
Population[['Country']] = Population[['Country']].replace("North Macedonia", "Republic of North Macedonia")
Population[['Country']] = Population[['Country']].replace("Russia", "Russian Federation")
Population[['Country']] = Population[['Country']].replace("Syria", "Syrian Arab Republic")

Sample_Size = 1000000

Population_Merged = pd.merge(ISO_toMerge,Population,  on ='Country',  how ='inner')

Population_Merged[['Population Perc']] = Population_Merged[['Population (2020)']]/Population_Merged[['Population (2020)']].sum()

Population_Merged = pd.merge(Population_Merged,Aggregated,  on ='Country',  how ='inner')

Population_Merged[['Sample size']] = Population_Merged['Population Perc']  / Population_Merged['name']*Population_Merged['name'].sum()*Sample_Size

Population_toMerge = Population_Merged.loc[:, Population_Merged.columns.intersection(['Country','Sample size'])]

df = pd.merge(df,Population_toMerge,  on ='Country',  how ='inner')

Total_Probability = df[['Sample size']].sum()

df[['Sample size']] = df[['Sample size']] / Total_Probability

vec = df[['Sample size']]

df_sampled = df.sample(n=Sample_Size, weights='Sample size',random_state=Random_seed)

gc.collect()

Briefly to sum up what I did:
* The NaN's were cleaned
* I estimated elevation which had almost 85% NaNs by approximation by use of location
* Next, I translated names of countries to more readable
* Last and most important, I found that distribution of records do not depict real-world density. I weighted the number of spots by population of the given country

# 3.How are the biggest cities distributed?

The data, I observe, contains 11 million so-called spots/places. First, it is improtant to understand what it realls means. And then for example it can be a village, city, but aso country or even continent. To understand it I plotted records with population bigger than 10.000.000 what showed me biggest cities, but also countries and continents. On the basis of it I can define them by use of appended codes.

In [ ]:
df_countries = df[df['feature code']=="PCLI"]
df_admin = df[df['feature code']=="ADM1"]
df_regions = df[df['feature code']=="RGN"]
df_continents = df[df['feature code']=="CONT"]
df_nature = df[df['feature code']=="AREA"]

df_administrative = df[df['feature class']=="A"]
df_hydrographic = df[df['feature class']=="H"]
df_area = df[df['feature class']=="L"]
df_populated = df[df['feature class']=="P"]
df_road = df[df['feature class']=="R"]
df_spot = df[df['feature class']=="S"]
df_administrative = df[df['feature class']=="T"]
df_undersea = df[df['feature class']=="U"]
df_vegetation = df[df['feature class']=="V"]

df['feature class'].value_counts()

Variable 'feature code' is very granular and has 100+ levels, that's too much. I decided to differentiate records by use of 'feature class'. Let's plot them together: 

In [ ]:
df_ten_and_more = df_populated[df.population>10000000]
df_five_ten = df_populated[(df.population<10000000) & (df.population>5000000)]
df_one_five = df_populated[(df.population<5000000) & (df.population>1000000)]
df_hundred_ones = df_populated[(df.population<1000000) & (df.population>100000)]

In [ ]:
plt.figure(1, figsize=(15,8))
m1 = Basemap(projection='merc',llcrnrlat=-60,urcrnrlat=65,llcrnrlon=-180,urcrnrlon=180,
             lat_ts=0,resolution='c')

m1.fillcontinents(color='#191919',lake_color='white') 
m1.drawmapboundary(fill_color='white')                
m1.drawcountries(linewidth=0.3, color="w")              

# Plot the data
mxy_4 = m1(df_hundred_ones["longitude"].tolist(), df_hundred_ones["latitude"].tolist())
A = m1.scatter(mxy_4[0], mxy_4[1], c="blue", lw=0.001, alpha=0.8, zorder=5, marker='*')
A

mxy_3 = m1(df_one_five["longitude"].tolist(), df_one_five["latitude"].tolist())
B = m1.scatter(mxy_3[0], mxy_3[1], c="green", lw=0.1, alpha=1, zorder=5, marker='o')
B

mxy_2 = m1(df_five_ten["longitude"].tolist(), df_five_ten["latitude"].tolist())
C = m1.scatter(mxy_2[0], mxy_2[1], c="yellow", lw=0.3, alpha=1, zorder=5, marker='p')
C

mxy_1 = m1(df_ten_and_more["longitude"].tolist(), df_ten_and_more["latitude"].tolist())
D = m1.scatter(mxy_1[0], mxy_1[1], c="red", lw=0.4, alpha=1, zorder=5, marker='s')
D

plt.legend((D,C,B,A),('>10mln','5mln-10mln', '1mln-5mln', '100k-1mln'),numpoints=1, loc=1)

plt.title("Big cities of the world")
plt.show()

In [ ]:
del df
del Population_Merged
del Aggregated
gc.collect()

**Finally, the question was, how the biggest cities are dispersed in the world?**

**I have to admit that classification of the city's population was a bit discussable. For example, Jakarta, Cairo and Lagos were classified as cities smaller than 10 mln. It is due to the fact that indeed their metropolitan area goes far more than 10mln but they as cities are limited to central areas. What is interesting: we observe huge number of cities bigger than 100k in Europe, but barely any bigger than 5mln. Opposite behaviour is observed in Brazil and North America. India, Japan and China have both huge number of cities bigger than 100k and mega-metropolies bigger than 5mln.**

# 4.What is continent according to DBSCAN?

Are continents differentiated reasonably or they are just humans' choice? Let's check it! For this, I ask smart guy, density-based algortihm ([more in my notebook](https://www.kaggle.com/jjmewtw/clustering-k-means-hierarchical-debscan-ema)) how (s)he would do it.

In [ ]:
df_limited = df_sampled[['longitude','latitude']].sample(n=50000,random_state=Random_seed)
df_DBSCAN = StandardScaler().fit_transform(df_limited)

DBcluster= DBSCAN(min_samples = 200,eps=0.4)
DBcluster_fit = DBcluster.fit(df_DBSCAN)
core_samples_mask = np.zeros_like(DBcluster_fit.labels_, dtype=bool)
core_samples_mask[DBcluster_fit.core_sample_indices_] = True

DBlabels = DBcluster_fit.labels_ 
DB_n_clusters_ = len(set(DBlabels)) 
DB_n_noise_ = list(DBlabels).count(-1)

print('Estimated number of clusters: %d' % DB_n_clusters_)
print('Estimated number of noise points: %d' % DB_n_noise_)

In [ ]:
df_limited = df_limited.reset_index()
df_cluster = pd.DataFrame(DBlabels,columns=['Cluster'])

df_limited = pd.concat([df_limited, df_cluster.reindex(df_limited.index)], axis=1)

In [ ]:
plt.figure(1, figsize=(15,8))
m5 = Basemap(projection='merc',llcrnrlat=-60,urcrnrlat=65,llcrnrlon=-180,urcrnrlon=180,
             lat_ts=0,resolution='c')

m5.fillcontinents(color='#191919',lake_color='white') 
m5.drawmapboundary(fill_color='white')                
m5.drawcountries(linewidth=0.3, color="w")

m5.drawparallels(
    np.arange(-60, 65, 2.),
    color = 'black', linewidth = 0.2,
    labels=[False, False, False, False])
m5.drawmeridians(
    np.arange(-180, 180, 2.),
    color = '0.25', linewidth = 0.2,
    labels=[False, False, False, False])

m5.drawmapscale(150., -50., -10, -55,2500,units='km', fontsize=10,yoffset=None,barstyle='fancy', labelstyle='simple',
    fillcolor1='w', fillcolor2='#000000',fontcolor='#000000',zorder=5)

unique_labels = set(DBlabels)
colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]
          
core_samples_mask = np.zeros_like(DBlabels, dtype=bool)
core_samples_mask[DBcluster.core_sample_indices_] = True

# Plot the data
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]
          
    class_member_mask = (DBlabels == k)

    xy = df_limited[class_member_mask & core_samples_mask]
    mxy = m5(xy["longitude"].tolist(), xy["latitude"].tolist())
    m5.scatter(mxy[0], mxy[1], c=tuple(col), lw=0.3, alpha=1, zorder=5, marker='p')

plt.title("Continents according to very strict DBSCAN with Euclidean metric and minimal sample: 300")
plt.show()

Alright, so according to this un-tuned DBSCAN there are only two continents: Americas and EurAfSia. Definitely, in this case water was at once ignored and not ignored. I mean, from one point of view I did not pre-define water as obstacle, but on other hand, anyway there are no cities in the water so type of obstacle is naturally built.

In [ ]:
df_limited = df_sampled[['longitude','latitude']].sample(n=50000,random_state=Random_seed)
df_DBSCAN = StandardScaler().fit_transform(df_limited)

DBcluster= DBSCAN(eps=0.15,min_samples = 170,metric="canberra")

DBcluster_fit = DBcluster.fit(df_DBSCAN)
core_samples_mask = np.zeros_like(DBcluster_fit.labels_, dtype=bool)
core_samples_mask[DBcluster_fit.core_sample_indices_] = True

DBlabels = DBcluster_fit.labels_ 
DB_n_clusters_ = len(set(DBlabels)) 
DB_n_noise_ = list(DBlabels).count(-1)

df_limited = df_limited.reset_index()
df_cluster = pd.DataFrame(DBlabels,columns=['Cluster'])
df_limited = pd.concat([df_limited, df_cluster.reindex(df_limited.index)], axis=1)

plt.figure(1, figsize=(15,8))
m5 = Basemap(projection='merc',llcrnrlat=-60,urcrnrlat=65,llcrnrlon=-180,urcrnrlon=180,
             lat_ts=0,resolution='c')

m5.fillcontinents(color='#191919',lake_color='white') 
m5.drawmapboundary(fill_color='white')                
m5.drawcountries(linewidth=0.3, color="w")

m5.drawparallels(
    np.arange(-60, 65, 2.),
    color = 'black', linewidth = 0.2,
    labels=[False, False, False, False])
m5.drawmeridians(
    np.arange(-180, 180, 2.),
    color = '0.25', linewidth = 0.2,
    labels=[False, False, False, False])

m5.drawmapscale(150., -50., -10, -55,2500,units='km', fontsize=10,yoffset=None,barstyle='fancy', labelstyle='simple',
    fillcolor1='w', fillcolor2='#000000',fontcolor='#000000',zorder=5)

unique_labels = set(DBlabels)
colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]
          
core_samples_mask = np.zeros_like(DBlabels, dtype=bool)
core_samples_mask[DBcluster.core_sample_indices_] = True

# Plot the data
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]
          
    class_member_mask = (DBlabels == k)

    xy = df_limited[class_member_mask & core_samples_mask]
    mxy = m5(xy["longitude"].tolist(), xy["latitude"].tolist())
    m5.scatter(mxy[0], mxy[1], c=tuple(col), lw=0.3, alpha=1, zorder=5, marker='p')
    
plt.title("Continents according to DBSCAN with canberra metric")
plt.show()

Alright, this DBSCAN is closer to agreement with modern humankind. The huge difference comes here from the choice of metric, in this case it is Canberra metric which is a weighted version of Mahattan one. It gave me 6 continents, Europe incorporated northern part of Africa. Asia was divided into two parts.

In [ ]:
df_limited = df_sampled[['longitude','latitude']].sample(n=50000,random_state=Random_seed)
df_DBSCAN = StandardScaler().fit_transform(df_limited)

DBcluster= DBSCAN(metric="l2",eps=0.14,min_samples = 120)

DBcluster_fit = DBcluster.fit(df_DBSCAN)
core_samples_mask = np.zeros_like(DBcluster_fit.labels_, dtype=bool)
core_samples_mask[DBcluster_fit.core_sample_indices_] = True

DBlabels = DBcluster_fit.labels_ 
DB_n_clusters_ = len(set(DBlabels)) 
DB_n_noise_ = list(DBlabels).count(-1)

df_limited = df_limited.reset_index()
df_cluster = pd.DataFrame(DBlabels,columns=['Cluster'])
df_limited = pd.concat([df_limited, df_cluster.reindex(df_limited.index)], axis=1)

plt.figure(1, figsize=(15,8))
m5 = Basemap(projection='merc',llcrnrlat=-60,urcrnrlat=65,llcrnrlon=-180,urcrnrlon=180,
             lat_ts=0,resolution='c')

m5.fillcontinents(color='#191919',lake_color='white') 
m5.drawmapboundary(fill_color='white')                
m5.drawcountries(linewidth=0.3, color="w")              

unique_labels = set(DBlabels)
colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]
          
core_samples_mask = np.zeros_like(DBlabels, dtype=bool)
core_samples_mask[DBcluster.core_sample_indices_] = True

m5.drawparallels(
    np.arange(-60, 65, 2.),
    color = 'black', linewidth = 0.2,
    labels=[False, False, False, False])
m5.drawmeridians(
    np.arange(-180, 180, 2.),
    color = '0.25', linewidth = 0.2,
    labels=[False, False, False, False])

m5.drawmapscale(150., -50., -10, -55,2500,units='km', fontsize=10,yoffset=None,barstyle='fancy', labelstyle='simple',
    fillcolor1='w', fillcolor2='#000000',fontcolor='#000000',zorder=5)

# Plot the data
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]
          
    class_member_mask = (DBlabels == k)

    xy = df_limited[class_member_mask & core_samples_mask]
    mxy = m5(xy["longitude"].tolist(), xy["latitude"].tolist())
    m5.scatter(mxy[0], mxy[1], c=tuple(col), lw=0.3, alpha=1, zorder=5, marker='p')
    
plt.title("Continents according to DBSCAN with L2 metric")
plt.show()

Here we received independent Africa from Eurasia. But South America was divided into multiple areas. Due is due to low density of population in the Amazonia, which divded ne continent into several regions.

**Finally, the question was, is continent arbitrary definition?**

**Our algorithm's answer is: definitely, continent's definition is arbitrary choice. But on other hand, even mathematical algorithms can't decide what would be the best defnitition. Some observations are clear: almost always Eurasia goes together, Africa often join them. Australia due to its distance likes to proclaim independence. Americas are always independent but they also prefer to go together. Further if we want to exclude Africa from Eurasia, then Americas will usually split to couple of regions as well.**

# 5.How to approximate elevation and what we can read from it?

What I did in the chapter '2' is approximation of the elevation by neighbouring valus. It was needed due to poor data population for elevation column. How did I approach this? I rounded latitude and longitude and imposed its average values over all NA's in the region.

In [ ]:
norm = Normalize()

df_map = df_sampled.sample(n=10000, random_state=Random_seed)

plt.figure(1, figsize=(15,8))
m2 = Basemap(projection='merc',llcrnrlat=-60,urcrnrlat=65,llcrnrlon=-180,urcrnrlon=180,lat_ts=0,resolution='c')

m2.fillcontinents(color='#C0C0C0', lake_color='#7093DB') 
m2.drawmapboundary(fill_color='white')                
m2.drawcountries(linewidth=.75, linestyle='solid', color='#000073',antialiased=True,zorder=3) 

m2.drawparallels(
    np.arange(-60, 65, 2.),
    color = 'black', linewidth = 0.2,
    labels=[False, False, False, False])
m2.drawmeridians(
    np.arange(-180, 180, 2.),
    color = '0.25', linewidth = 0.2,
    labels=[False, False, False, False])

x,y = m2(*(df_map.longitude.values, df_map.latitude.values))

numcols, numrows = 1000, 1000
xi = np.linspace(x.min(), x.max(), numcols)
yi = np.linspace(y.min(), y.max(), numrows)
xi, yi = np.meshgrid(xi, yi)

x_1, y_1, z_1 = df_map.longitude.values, df_map.latitude.values, df_map.elevation_y.values
zi = griddata((x,y),df_map.elevation_y.values,(xi, yi),method='linear')

con = m2.contourf(xi, yi, zi, zorder=4, alpha=0.7, cmap='RdPu')

m2.scatter(x,y,color='#545454',edgecolor='#ffffff',alpha=.75,s=50 * norm(df_map.elevation_y),cmap='RdPu',vmin=zi.min(), vmax=zi.max(), zorder=4)

m2.drawmapscale(150., -50., -10, -55,2500,units='km', fontsize=10,yoffset=None,barstyle='fancy', labelstyle='simple',
    fillcolor1='w', fillcolor2='#000000',fontcolor='#000000',zorder=5)

cbar = plt.colorbar(con, orientation='horizontal', fraction=.057, pad=0.05)
cbar.set_label("Regional elevation - m")

plt.title("Elevation of the region with mountains marked")
plt.show()

**Finally: the question was: How to approximate elevation and what we can read from it?**

**First, please pay attention that this map was built by use of approximated elevation, namely 85% of the data was preliminarly in NA class. I would say it is quite good result for this. We easily can observe the most relevant mountain ranges of the wrold with generally plotted elevation along. I can say that application of rounded longitude/latitude with apporximation to full degree is absolutely enough to receive reasonable results.**